In [17]:
%matplotlib

Using matplotlib backend: MacOSX


In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [19]:
# import weather data
weather_file = pd.read_csv("weather2018.csv")
weather_file = weather_file.rename(columns={"DATE": "Date",
                                            "AWND": "Average Wind Speed",
                                            "PRCP": "Precipitation",
                                            "SNOW": "Snow",
                                            "TAVG": "Average Temperate"})
weather_file.head()

,STATION,NAME,Date,Average Wind Speed,Precipitation,Snow,Average Temperate
0,USW00014739,"BOSTON, MA US",2018-01-01,16.33,0.00,0.0,6
1,USW00014739,"BOSTON, MA US",2018-01-02,12.75,0.00,0.0,10
2,USW00014739,"BOSTON, MA US",2018-01-03,9.40,0.00,0.0,21
3,USW00014739,"BOSTON, MA US",2018-01-04,22.59,1.35,13.4,26
4,USW00014739,"BOSTON, MA US",2018-01-05,24.83,0.00,0.0,18


In [20]:
#use for loop merge ridership
total_rider = ['Weekday_Ridership_2018_01_Jan.csv',
               'Weekday_Ridership_2018_02_Feb.csv',
        'Weekday_Ridership_2018_03_Mar.csv','Weekday_Ridership_2018_04_Apr.csv',
        'Weekday_Ridership_2018_05_May.csv','Weekday_Ridership_2018_06_Jun.csv',
        'Weekday_Ridership_2018_07_Jul.csv','Weekday_Ridership_2018_08_Aug.csv',
        'Weekday_Ridership_2018_09_Sep.csv','Weekday_Ridership_2018_10_Oct.csv',
        'Weekday_Ridership_2018_11_Nov.csv','Weekday_Ridership_2018_12_Dec.csv',]
ridership_df = pd.DataFrame({})
for file in total_rider:
    tmp = pd.read_csv('Weekday_Ridership_2018/' + file)
    ridership_df = ridership_df.append(tmp)
    
#column change
ridership_df = ridership_df.rename(columns={"date":"Date","MODE":"Mode",
                                            "RIDERSHIP":"Ridership",
                                            "ROUTE_OR_LINE":"Route"})

#column del
del ridership_df["COUNTOFDATES"]
del ridership_df["AVG_RIDERSHIP"]

#correct dtype
ridership_df['Date'] = ridership_df['Date'].astype('datetime64[ns]')
ridership_df['Date'] = pd.to_datetime(ridership_df['Date'])
#create a conditional column
ridership_df['Season'] =np.where((ridership_df['Date'] > '2018-05-01')& (ridership_df['Date'] <= '2018-08-01'),'Summer',
                                  np.where((ridership_df['Date'] > '2018-08-01')& (ridership_df['Date'] <= '2018-11-01'),'Fall', 
                                           np.where((ridership_df['Date'] > '2018-11-01') | (ridership_df['Date'] <= '2018-02-01'),'Winter',
                                                    'Spring')))
ridership_df

,Date,Mode,Ridership,Route,Season
0,2018-01-01,Bus,6.788939e+06,Bus,Winter
1,2018-01-01,Bus,6.351820e+05,Silver Line,Winter
2,2018-01-01,Rail,1.323204e+06,Blue Line,Winter
3,2018-01-01,Rail,3.141136e+06,Green Line,Winter
4,2018-01-01,Rail,3.987313e+06,Orange Line,Winter
...,...,...,...,...,...
1,2018-12-01,Bus,6.716490e+05,Silver Line,Winter
2,2018-12-01,Rail,1.303504e+06,Blue Line,Winter
3,2018-12-01,Rail,2.791420e+06,Green Line,Winter
4,2018-12-01,Rail,3.687113e+06,Orange Line,Winter


In [21]:
#find sessional ridership
season_rider =ridership_df.groupby(['Season']).sum()['Ridership']
season_rider

Season
Fall      7.063330e+07
Spring    6.918902e+07
Summer    6.937518e+07
Winter    5.996839e+07
Name: Ridership, dtype: float64

In [22]:
# Labels for the sections of our pie chart
labels = ["Fall", "Spring", "Summer", "Winter"]
# The values of each section of the pie chart
sizes = [(season_rider['Fall'], season_rider['Spring'], season_rider['Summer'],season_rider['Winter'])]
# The colors of each section of the pie chart
colors = ["red", "orange", "lightcoral", "lightskyblue"]
# Tells matplotlib to seperate the "Python" section from the others
explode = (0.1, 0, 0, 0)

plt.pie(sizes, explode=explode, labels=labels, colors=colors,
        autopct="%1.1f%%", shadow=True, startangle=140)
plt.title('Seaonal Ridership Performance')

plt.savefig("PyPies.png",bbox_inches="tight")
plt.show()

/Users/jane/opt/anaconda3/envs/pandas_test/lib/python3.7/site-packages/ipykernel_launcher.py:11: MatplotlibDeprecationWarning: Non-1D inputs to pie() are currently squeeze()d, but this behavior is deprecated since 3.1 and will be removed in 3.3; pass a 1D array instead.
  # This is added back by InteractiveShellApp.init_path()


In [23]:
# total yearly ridership according to diff route
route_rider =ridership_df.groupby(['Route']).sum()['Ridership']
route_rider

Route
Blue Line      1.834756e+07
Bus            8.647179e+07
Green Line     4.135930e+07
Orange Line    5.183919e+07
Red Line       6.211576e+07
Silver Line    9.032315e+06
Name: Ridership, dtype: float64

In [24]:
ridership_df['Route'].unique()

array(['Bus', 'Silver Line', 'Blue Line', 'Green Line', 'Orange Line',
       'Red Line'], dtype=object)

In [25]:
#withdraw routes data
ridership = ridership_df.set_index('Date')
ridership

,Mode,Ridership,Route,Season
Date,,,,
2018-01-01,Bus,6.788939e+06,Bus,Winter
2018-01-01,Bus,6.351820e+05,Silver Line,Winter
2018-01-01,Rail,1.323204e+06,Blue Line,Winter
2018-01-01,Rail,3.141136e+06,Green Line,Winter
2018-01-01,Rail,3.987313e+06,Orange Line,Winter
...,...,...,...,...
2018-12-01,Bus,6.716490e+05,Silver Line,Winter
2018-12-01,Rail,1.303504e+06,Blue Line,Winter
2018-12-01,Rail,2.791420e+06,Green Line,Winter


In [28]:
#collect route data
bus_data = ridership[ridership['Route'] == 'Bus']['Ridership']
silver_data = ridership[ridership['Route'] == 'Silver Line']['Ridership']
blue_data =ridership[ridership['Route'] == 'Blue Line']['Ridership']
green_data =ridership[ridership['Route'] == 'Green Line']['Ridership']
orange_data =ridership[ridership['Route'] == 'Orange Line']['Ridership']
red_data =ridership[ridership['Route'] == 'Red Line']['Ridership']

months = range(1,13)
#months
# Plot different line chart

#plt.plot(months, bus_data, color="black", label="Bus" )
bus, = plt.plot(months,bus_data/10000, color="Black", label="Bus")
silver, = plt.plot(months,silver_data/10000, color="Pink", label="Sliver Line")
blue, = plt.plot(months,blue_data/10000, color="blue", label="Blue Line")
green, = plt.plot(months,green_data/10000, color="green", label="Green Line")
orange, = plt.plot(months,orange_data/10000, color="orange", label="Orange Line")
red, = plt.plot(months,red_data/10000, color="red", label="Red Line")
plt.xticks(months)
plt.legend(loc="best")
plt.grid()

#edit name
plt.title("2018 Routes' Ridership Performance")
plt.xlabel("Months")
plt.ylabel("Ridership (Ten thousand)")
#plt.rcParams["figure.figsize"] =[6,10]
plt.savefig("2018 Ridership Performance.png")
plt.show()

In [27]:
weather_file.dtypes
weather_file['Date'] = weather_file['Date'].astype('datetime64[ns]')

grouped = weather_file.groupby(weather_file['Date'].dt.strftime('%B')).agg({'Average Wind Speed':'mean', 'Precipitation': 'mean', 
                                            'Snow':'mean','Average Temperate':'mean'})

#how to calculate an individual column
#average_rain =weather_file.groupby(weather_file['Date'].dt.strftime('%B'))['Precipitation'].mean().sort_values()

grouped_weather = grouped.reset_index()
#grouped_weather['Session'] = np.where(grouped_weather['Date'].isin(['June','July','August']),'Summer',
                             #np.where(grouped_weather['Date'].isin(['September','October','November']),'Fall',
                             #np.where(grouped_weather['Date'].isin(['December','January','February']),'Winter','Spring'
                              #       )))

categories =['January', 'February', 'March','April','May','June','July','August','September','October','November','December']
grouped_weather['Date'] = pd.Categorical(grouped_weather['Date'], categories, ordered=True)
grouped_weather =grouped_weather.sort_values('Date') 
#grouped_weather = grouped_weather.set_index('Date')
grouped_weather =grouped_weather.set_index('Date')
grouped_weather

,Average Wind Speed,Precipitation,Snow,Average Temperate
Date,,,,
January,13.327742,0.158710,0.574194,28.580645
February,11.081071,0.134643,0.296429,38.107143
March,15.089032,0.163548,0.751613,36.741935
April,11.938333,0.154000,0.043333,44.700000
May,10.001613,0.061290,0.000000,60.870968
June,10.253333,0.098667,0.000000,67.366667
July,9.870968,0.146774,0.000000,75.870968
August,9.466129,0.150000,0.000000,76.935484
September,9.656333,0.170667,0.000000,68.366667


In [ ]:

#grouped_weather['Average Wind Speed'] = grouped_weather['Average Wind Speed'].astype('datetime64[ns]')
#grouped_weather['Precipitation'] = grouped_weather['Precipitation'].astype('datetime64[ns]')
#grouped_weather['Snow'] = grouped_weather['Snow'].astype('datetime64[ns]')
#grouped_weather['Average Temperate'] = grouped_weather['Average Temperate'].astype('datetime64[ns]')
#grouped_weather.dtypes

In [ ]:
#collect weather data
wind_data = grouped_weather['Average Wind Speed']
rain_data = grouped_weather['Precipitation']
snow_data =grouped_weather['Snow']
tem_data =grouped_weather['Average Temperate']

months = range(1,13)

#wind_data
wind =plt.plot(months,wind_data, color="Black", label="Wind")
plt.xticks(months)
plt.grid()

#edit name
plt.title("2018 Wind Speed Performance")
plt.xlabel("Months")
plt.ylabel("Wind Speed (km/h)")
#plt.rcParams["figure.figsize"] =[8,3]
plt.savefig("Wind Speed Performance.png",bbox_inches="tight")
plt.show()

In [ ]:
rain =plt.plot(months,rain_data, color="Red", label="Rain")
plt.xticks(months)
plt.grid()

#edit name
plt.title("2018 Precipitation Performance")
plt.xlabel("Months")
plt.ylabel("Precipitation (inches)")
plt.savefig("Precipitation Performance.png",bbox_inches="tight")
plt.show()

In [ ]:
snow =plt.plot(months,snow_data, color="Green", label="Snow")
plt.xticks(months)
plt.grid()
plt.rcParams["figure.figsize"] =[8,3]

plt.title("2018 Snow Performance")
plt.xlabel("Months")
plt.ylabel("Snow (inches)")
plt.savefig("Snow Performance.png",bbox_inches="tight")
plt.show()

In [ ]:
temp =plt.plot(months,tem_data, color="Yellow", label="Temperature")
plt.xticks(months)
plt.grid()

plt.title("2018 Temperature")
plt.xlabel("Months")
plt.ylabel("Temperature (F)")
plt.savefig("2018 Temperature.png",bbox_inches="tight")
plt.show()

# How does ridership change each season?

* By looking at “Seasonal Ridership Performance”, we can see Winter’s Total Ridership is 3% to 4% less than other seasons. Fall has the most ridership, Summer and Spring only have 0.1% of difference.

* Therefore, Fall has the most ridership, but when season transfer to Winter, the ridership become the least.

# Under which weather circumstance in 2018, MBTA has the best   ridership performance?

* After comparing each Route’s performance by plot our data into Multi Line, we can see each ridership number is very different with each other, but their line patterns are almost the same. October always has the best ridership performance in 2018.
* Then, I look into the monthly weather performance, and plot each of the factors into line chart. 
* Therefore, In October 2018, when there is no snow, the average temperature is 54 Fahrenheit, the average wind speed is 11km/h, and the average precipitation is 0.122 inches, , MBTA has the best ridership performance.


